In [ ]:
#|hide
#|default_exp export

# nbprocess.export
- Exporting a notebook to a library

In [ ]:
#|export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.imports import *

from fastcore.script import *
from fastcore.imports import *
from fastcore.xtras import *

from collections import defaultdict
from pprint import pformat
from inspect import signature,Parameter
import ast,contextlib,copy

In [ ]:
from fastcore.test import *
from pdb import set_trace
from importlib import reload
import shutil

Special comments at the start of a cell can be used to provide information to `nbprocess` about how to process a cell, so we need to be able to find the location of these comments.

In [ ]:
minimal = read_nb('../tests/minimal.ipynb')

In [ ]:
#|export
def _directive(s):
    s = (s.strip()[2:]).strip().split()
    if not s: return None
    direc,*args = s
    return direc,args

In [ ]:
#|export
def extract_directives(cell):
    "Take leading comment directives from lines of code in `ss`, remove `#|`, and split"
    ss = cell.source.splitlines(True)
    first_code = first(i for i,o in enumerate(ss) if not o.strip() or not re.match(r'\s*#\|', o))
    if not ss or first_code==0: return {}
    cell['source'] = ''.join(ss[first_code:])
    res = L(_directive(s) for s in ss[:first_code]).filter()
    return {k:v for k,v in res}

Comment directives start with `#`, followed by whitespace delimited tokens, which `extract_directives` extracts from the start of a cell, up until a blank line or a line containing something other than comments. The extracted lines are removed from the source.

In [ ]:
exp  = AttrDict(source = """#|export module
#| hide
1+2
#bar""")
test_eq(extract_directives(exp), dict(export=['module'],hide=[]))
test_eq(exp.source, "1+2\n#bar")

In [ ]:
#|export
def opt_set(var, newval):
    "newval if newval else var"
    return newval if newval else var

In [ ]:
#|export
class NBProcessor:
    "Process cells and nbdev comments in a notebook"
    def __init__(self, path=None, procs=None, preprocs=None, postprocs=None, nb=None, debug=False):
        self.nb = read_nb(path) if nb is None else nb
        self.procs,self.preprocs,self.postprocs = map(L, (procs,preprocs,postprocs))
        self.debug = debug

    def _process_cell(self, cell):
        self.cell = cell
        cell['_directives'] = extract_directives(cell)
        for proc in self.procs:
            if cell.cell_type=='code':
                for cmd,args in cell._directives.items(): self._process_comment(proc, cmd, args)
            if callable(proc): cell = opt_set(cell, proc(cell))

    def _process_comment(self, proc, cmd, args):
        f = getattr(proc, f'_{cmd}_', None)
        if not f: return True
        if self.debug: print(cmd, args, f)
        return f(self, *args)
        
    def process(self):
        "Process all cells with `process_cell`"
        for proc in self.preprocs: self.nb = opt_set(self.nb, proc(self.nb))
        for i in range_of(self.nb.cells): self._process_cell(self.nb.cells[i])
        self.nb.cells = [c for c in self.nb.cells if c and getattr(c,'source',None) is not None]
        for proc in self.postprocs: self.nb = opt_set(self.nb, proc(self.nb))

Cell processors can be callables (e.g regular functions), in which case they are called for every cell:

In [ ]:
everything_fn = '../tests/01_everything.ipynb'

def print_execs(cell):
    if 'exec' in cell.source: print(cell.source)

NBProcessor(everything_fn, print_execs).process()

exec("o_y=1")
exec("p_y=1")
_all_ = [o_y, 'p_y']


Comment directives are put in a cell attribute `_directive` as a dictionary keyed by directive name:

In [ ]:
def printme_func(cell):
    if 'printme' in cell._directives: print(cell._directives['printme'])

NBProcessor(everything_fn, printme_func).process()

['testing']


However, a more convenient way to handle comment directives is to use a *class* as a processor, and include a method in your class with the same name as your directive, surrounded by underscores:

In [ ]:
class _PrintExample:
    def _printme_(self, nbp, to_print): print(to_print)

everything_fn = '../tests/01_everything.ipynb'
npb = NBProcessor(everything_fn, _PrintExample())
npb.process()

testing


## Export -

In [ ]:
#|skip
basic_export_nb2('00_read.ipynb', 'read')
basic_export_nb2('01_maker.ipynb', 'maker')
basic_export_nb2('02_process.ipynb', 'process')

g = exec_new('import nbprocess.process')
assert hasattr(g['nbprocess'].process, 'NBProcessor')